In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv(r"C:\Users\SURABHI\Downloads\Netflix-1\combined_data_1.txt\combined_data_1.txt", header=None, names=['Cust_Id','Rating'],usecols=[0,1])
df.head()

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


In [5]:
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan.head()

,Rating
0,True
1,False
2,False
3,False
4,False


In [7]:
df_nan = df_nan[df_nan['Rating'] == True]
df_nan.shape

(4499, 1)

In [9]:
df_nan=pd.DataFrame(df['Rating'].isnull())
df_nan=df_nan[df_nan['Rating']==True]
df_nan

,Rating
0,True
548,True
694,True
2707,True
2850,True
...,...
24046714,True
24047329,True
24056849,True
24057564,True


In [11]:
p = df.groupby('Rating')['Rating'].agg(['count'])
p

,count
Rating,
1.0,1118186
2.0,2439073
3.0,6904181
4.0,8085741
5.0,5506583


In [13]:
df_nan = df_nan.reset_index()
df_nan.head()

,index,Rating
0,0,True
1,548,True
2,694,True
3,2707,True
4,2850,True


In [15]:
movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1
    
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

print(f'Movie numpy: {movie_np}')
print(f'Length: {len(movie_np)}')

Movie numpy: [1.000e+00 1.000e+00 1.000e+00 ... 4.499e+03 4.499e+03 4.499e+03]
Length: 24053764


In [17]:
df = df[pd.notnull(df['Rating'])]
df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] =df['Cust_Id'].astype(int)
print('-Dataset examples-')
df.head()

-Dataset examples-


,Cust_Id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [19]:
f = ['count','mean']

In [21]:
dataset_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)

dataset_movie_summary.index = dataset_movie_summary.index.map(int)

movie_benchmark = round(dataset_movie_summary['count'].quantile(0.7),0)

drop_movie_list = dataset_movie_summary[dataset_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

Movie minimum times of review: 1799.0


In [23]:
dataset_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
dataset_cust_summary.index = dataset_cust_summary.index.map(int)
cust_benchmark = round(dataset_cust_summary['count'].quantile(0.7),0)
drop_cust_list = dataset_cust_summary[dataset_cust_summary['count'] < cust_benchmark].index

print(f'Customer minimum times of review: {cust_benchmark}')

Customer minimum times of review: 52.0


In [25]:
print(f'Original Shape: {df.shape}')

Original Shape: (24053764, 3)


In [27]:
df = df[~df['Movie_Id'].isin(drop_movie_list)]
df = df[~df['Cust_Id'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(df.shape))

After Trim Shape: (17337458, 3)


In [28]:
print('-Data Examples-')
df.head()

-Data Examples-


,Cust_Id,Rating,Movie_Id
696,712664,5.0,3
697,1331154,4.0,3
698,2632461,3.0,3
699,44937,5.0,3
700,656399,4.0,3


In [31]:
df_title = pd.read_csv("C:\\Users\\SURABHI\\Downloads\\Netflix-1\\movie_titles.csv.csv", encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'],usecols=[0,1,2])
df_title.set_index('Movie_Id', inplace = True)

print (df_title.head(10))

            Year                          Name
Movie_Id                                      
1         2003.0               Dinosaur Planet
2         2004.0    Isle of Man TT 2004 Review
3         1997.0                     Character
4         1994.0  Paula Abdul's Get Up & Dance
5         2004.0      The Rise and Fall of ECW
6         1997.0                          Sick
7         1992.0                         8 Man
8         2004.0    What the #$*! Do We Know!?
9         1991.0      Class of Nuke 'Em High 2
10        2001.0                       Fighter


In [33]:
import math
import re
import matplotlib.pyplot as plt

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate,train_test_split

In [35]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:100000], reader)

# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9984  0.9933  1.0018  0.9978  0.0035  
MAE (testset)     0.7952  0.7912  0.8017  0.7960  0.0043  
Fit time          2.05    1.55    1.34    1.65    0.30    
Test time         0.49    0.41    0.29    0.40    0.08    


{'test_rmse': array([0.99838538, 0.99333449, 1.00179127]),
 'test_mae': array([0.79524422, 0.79117516, 0.80171427]),
 'fit_time': (2.0545341968536377, 1.5498549938201904, 1.3425116539001465),
 'test_time': (0.49497556686401367, 0.41201019287109375, 0.2949826717376709)}

In [36]:
merged_df=pd.merge(df,df_title,on='Movie_Id',how='inner')
merged_df.head(10)

,Cust_Id,Rating,Movie_Id,Year,Name
0,712664,5.0,3,1997.0,Character
1,1331154,4.0,3,1997.0,Character
2,2632461,3.0,3,1997.0,Character
3,44937,5.0,3,1997.0,Character
4,656399,4.0,3,1997.0,Character
5,439011,1.0,3,1997.0,Character
6,1644750,3.0,3,1997.0,Character
7,2031561,4.0,3,1997.0,Character
8,616720,4.0,3,1997.0,Character
9,2467008,4.0,3,1997.0,Character


In [39]:
dataset_712664 = merged_df[(merged_df['Cust_Id'] == 712664) & (merged_df['Rating'] == 5)]
dataset_712664.head(10)

,Cust_Id,Rating,Movie_Id,Year,Name
0,712664,5.0,3,1997.0,Character
215690,712664,5.0,79,1956.0,The Killing
411093,712664,5.0,175,1992.0,Reservoir Dogs
643954,712664,5.0,199,1978.0,The Deer Hunter
731826,712664,5.0,241,1959.0,North by Northwest
799580,712664,5.0,256,2000.0,Ghost Dog: The Way of the Samurai
1375336,712664,5.0,348,1988.0,The Last Temptation of Christ
1404815,712664,5.0,357,2003.0,House of Sand and Fog
1608913,712664,5.0,416,2003.0,Elephant
1668999,712664,5.0,442,1988.0,Mississippi Burning


<h2>Train SVD to predict movies for user with userId=1</h2>

In [42]:
surprise_data=Dataset.load_from_df(merged_df[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [43]:
svd=SVD()

In [46]:
train_set=surprise_data.build_full_trainset()

# Fitting the SVD algorithm (svd) to the training set (train_set)
svd.fit(train_set)

In [48]:
def get_top_movies(user_id, data ,algo):
    rating = data['Rating'].unique()
    top_movies = []
    
    rated_movie_ids = set(data[data['Cust_Id'] == user_id]['Movie_Id'])
    avg_ratings = data.groupby('Movie_Id')['Rating'].mean().to_dict()
    
    for rating in rating:
        rating_movie = data[data['Rating']==rating]['Movie_Id'].unique()
        
        unrated_movies = [movie_id for movie_id in rating_movie if movie_id not in rated_movie_ids]
        predictions = [algo.predict(user_id, movie_id) for movie_id in unrated_movies]
        
        predictions.sort(key=lambda x: x.est, reverse=True)
        
        if predictions:
            top_movie = predictions[0]
            
            movie_title= data[data['Movie_Id']==top_movie.iid]['Name'].iloc[0]

            avg_rating = avg_ratings.get(top_movie.iid, "No Rating")  
            top_movies.append({'Movie_Id': top_movie.iid,
                               'Name': movie_title,
                               'Rating': avg_rating,
                               'Estimated Score': top_movie.est})
            
            
    return pd.DataFrame(top_movies)

In [50]:
import random
user_ids = merged_df['Cust_Id'].unique()
User_Id = random.choice(user_ids)

top_movies = get_top_movies(User_Id, merged_df, svd)

top_movies = top_movies.sort_values(by='Estimated Score', ascending=False).reset_index(drop=True)

print(f"\033[1m Best Suited Movie For {User_Id}  ")
top_movies.head(1)


 Best Suited Movie For 1740218  


,Movie_Id,Name,Rating,Estimated Score
0,4145,Law & Order: Special Victims Unit: The First Year,4.15094,4.724529
